In [26]:
!python -V

Python 3.9.7


In [27]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [28]:
df = pd.read_parquet('../data/yellow_tripdata_2022-01.parquet')
print(df.dtypes)
print(df.shape)
print(len(df.columns))

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object
(2463931, 19)
19


In [24]:
df.head(1000)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,0 days 00:17:49
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,0 days 00:08:24
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,0 days 00:08:58
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,0 days 00:10:02
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,0 days 00:37:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,2022-01-01 00:15:15,2022-01-01 00:19:08,2.0,0.98,1.0,N,79,170,1,5.0,0.5,0.5,1.76,0.0,0.3,10.56,2.5,0.0,0 days 00:03:53
996,2,2022-01-01 00:32:25,2022-01-01 00:44:22,3.0,2.13,1.0,N,79,233,2,10.0,0.5,0.5,0.00,0.0,0.3,13.80,2.5,0.0,0 days 00:11:57
997,2,2022-01-01 00:53:58,2022-01-01 00:57:04,2.0,0.35,1.0,N,107,107,2,4.0,0.5,0.5,0.00,0.0,0.3,7.80,2.5,0.0,0 days 00:03:06
998,2,2022-01-01 00:21:20,2022-01-01 00:32:00,1.0,1.85,1.0,N,164,249,1,9.0,0.5,0.5,1.00,0.0,0.3,13.80,2.5,0.0,0 days 00:10:40


In [25]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

In [12]:
# Crear un objeto timedelta
td = pd.Timedelta(days=1, hours=0, minutes=0, seconds=0)

# Obtener el total de segundos en el timedelta
total_seconds = td.total_seconds()

# Imprimir el total de segundos
print(total_seconds)

86400.0


In [ ]:
df = df[(df.duration >= 1) & (df.duration <= 60)]